In [1]:
!pip install sagemaker --upgrade --quiet

In [2]:
import sagemaker, boto3, json
from sagemaker import get_execution_role

role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.serverless import ServerlessInferenceConfig
from sagemaker.serializers import DataSerializer

In [4]:
# create Hugging Face Model Class
# Hub Model configuration. <https://huggingface.co/models>
hub = {
    'HF_MODEL_ID':'DhruvJariwala/deepfake_vs_real_image_detection',
    'HF_TASK':'image-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,                      # configuration for loading model from Hub
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.28",  # transformers version used
   pytorch_version="2.0",        # pytorch version used
   py_version='py310',            # python version used
)


In [5]:
# create a serializer for the data
image_serializer = DataSerializer(content_type='image/x-image')

In [6]:
# Specify MemorySizeInMB and MaxConcurrency in the serverless config object
#serverless_config = ServerlessInferenceConfig(
#    memory_size_in_mb=2048, max_concurrency=10,
#)

# deploy the endpoint endpoint
#predictor = huggingface_model.deploy(
#    serverless_inference_config=serverless_config
#)

predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge",
    serializer=image_serializer,
)

---------!

In [7]:
!wget https://huggingface.co/datasets/hf-internal-testing/fixtures_ade20k/raw/main/ADE_val_00000001.jpg

--2024-05-09 10:13:10--  https://huggingface.co/datasets/hf-internal-testing/fixtures_ade20k/raw/main/ADE_val_00000001.jpg
Resolving huggingface.co (huggingface.co)... 18.161.125.35, 18.161.125.84, 18.161.125.127, ...
Connecting to huggingface.co (huggingface.co)|18.161.125.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52650 (51K) [image/jpeg]
Saving to: ‘ADE_val_00000001.jpg’

100%[======================================>] 52,650       271KB/s   in 0.2s   

2024-05-09 10:13:10 (271 KB/s) - ‘ADE_val_00000001.jpg’ saved [52650/52650]



In [8]:
image_path = "ADE_val_00000001.jpg"

with open(image_path, "rb") as data_file:
    image_data = data_file.read()
    res = predictor.predict(data=image_data)
    print(res)

[{'score': 0.5058110356330872, 'label': 'Fake'}, {'score': 0.49418893456459045, 'label': 'Real'}]


In [ ]:
# request
# predictor.predict(data)

In [9]:
predictor.delete_model()
predictor.delete_endpoint()